In [1]:
import os
import pandas as pd
import qdrant_client
from dotenv import load_dotenv
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from qdrant_client.http.models import VectorParams, Distance
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Load environment variables
load_dotenv()

QDRANT_HOST = os.getenv("QDRANT_HOST")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_COLLECTION_NAME = "imdb"

In [3]:
# Load IMDb dataset
df = pd.read_csv("C:/Users/saisu/Documents/Learning/RAG_project_imdb/data/imdb_top_1000.csv")


In [4]:
# Convert rows into LangChain Document format
documents = [
    Document(
        metadata={"title": row["Series_Title"], "year": row["Released_Year"], "genre": row["Genre"], "rating": row["IMDB_Rating"]},
        page_content=(
            f"Movie: {row['Series_Title']}, Released: {row['Released_Year']}, Genre: {row['Genre']}, "
            f"Rating: {row['IMDB_Rating']}, Director: {row['Director']},  Overview: {row['Overview']}"
            f"Starring: {row['Star1']}, {row['Star2']}, {row['Star3']}, {row['Star4']}."
        )
    )
    for _, row in df.iterrows()
]
documents[:5]

[Document(metadata={'title': 'The Shawshank Redemption', 'year': '1994', 'genre': 'Drama', 'rating': 9.3}, page_content='Movie: The Shawshank Redemption, Released: 1994, Genre: Drama, Rating: 9.3, Director: Frank Darabont,  Overview: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.Starring: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler.'),
 Document(metadata={'title': 'The Godfather', 'year': '1972', 'genre': 'Crime, Drama', 'rating': 9.2}, page_content="Movie: The Godfather, Released: 1972, Genre: Crime, Drama, Rating: 9.2, Director: Francis Ford Coppola,  Overview: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.Starring: Marlon Brando, Al Pacino, James Caan, Diane Keaton."),
 Document(metadata={'title': 'The Dark Knight', 'year': '2008', 'genre': 'Action, Crime, Drama', 'rating': 9.0}, page_content='Movie: The Dark Knight, Released: 2008, Genr

In [5]:
# Initialize Qdrant Client
client = qdrant_client.QdrantClient(url=QDRANT_HOST, api_key=QDRANT_API_KEY, timeout=120)

# Check if collection exists, then create it
if not client.collection_exists(QDRANT_COLLECTION_NAME):
    client.create_collection(
        collection_name=QDRANT_COLLECTION_NAME,
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )


In [6]:
# Initialize embeddings
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
embeddings = OpenAIEmbeddings()

# Connect vector store
vector_store = Qdrant(
    client=client,
    collection_name=QDRANT_COLLECTION_NAME,
    embeddings=embeddings
)

C:\Users\saisu\AppData\Local\Temp\ipykernel_31092\2627495524.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\saisu\AppData\Local\Temp\ipykernel_31092\2627495524.py:6: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vector_store = Qdrant(


In [7]:
# Define text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n"]
)

# Process documents and split text correctly
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)  # Use page_content instead of passing Document object
    chunked_documents.extend(chunks)

# Display some chunked samples
print(chunked_documents[:5])

['Movie: The Shawshank Redemption, Released: 1994, Genre: Drama, Rating: 9.3, Director: Frank Darabont,  Overview: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.Starring: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler.', "Movie: The Godfather, Released: 1972, Genre: Crime, Drama, Rating: 9.2, Director: Francis Ford Coppola,  Overview: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.Starring: Marlon Brando, Al Pacino, James Caan, Diane Keaton.", 'Movie: The Dark Knight, Released: 2008, Genre: Action, Crime, Drama, Rating: 9.0, Director: Christopher Nolan,  Overview: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.Starring: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine.', 'Movie: The Godfather: P

In [8]:
vector_store.add_texts(chunked_documents)

['2c06810383f64d67bda1c46d4fa1f988',
 '4715966a828c44848de3d6e8ec705a54',
 'eeaa9903a88040f7a4543f8c42bb733f',
 '65024aef0b094640b8453590e399ad77',
 '4ba75b0024b54fbcb4b8ce69824f868b',
 '20c8f683d6b94df0a54def5dd504289d',
 '7c3a11bbfb52418dae92a7560c364a4d',
 'ab80da909ebe4ae1b0dd7e7bb433fa68',
 '43f87d50428c4011a769db6ab9321e06',
 '2515cf98d00341c4b74cbdaf350308d4',
 '4e52363ca9414347a681aaf3ca64779d',
 '69bdfef33c704bcc82a8c6ce9ffdc990',
 '6ec1dfa9b63a4e09a8dd476ad29d9ee1',
 '04d46d4c3f25407586c82596b1c9a50a',
 '80710ae84d5840f4840d1bfc3f3f9584',
 'ec035965bfde422791392376e9b55982',
 '6140703a9732425e89f68400ef0f2584',
 '13a4694bb3734bf19c9e7903c29b3320',
 '5abf9cf5b60a4dc48abc7e33ac491609',
 'b0bbea7615ba450fa1f7e00b8ed9b689',
 'b1a3e52b2b82427886a4ed2329aa2a7e',
 '0e4f844ff2da4399aea6209641190042',
 '7bdcc068c5f3475c91087bd4688033e8',
 'e50d63424109431aa8f535c2096bde3a',
 'c3e676e1705647588f73ef8f7d6b8616',
 'a66a363271bc48ff9a965504695d9a25',
 'fc11b40f58a342d8b3cda8f0eaa64d6c',
 

In [9]:
# plug vector store into retrieval chain

from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

qa=RetrievalQA.from_chain_type(
    llm=ChatGroq(model='deepseek-r1-distill-llama-70b', temperature=0.5),
    chain_type='stuff',
    retriever=vector_store.as_retriever()
)

In [10]:
import re

def remove_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

In [11]:
def get_response(query):
    response=qa.run(query)
    if "<think>" in response:
        response=remove_think_tags(response)
    return response

In [12]:
query="Inception, who are actors in it?"
print(get_response(query=query))

C:\Users\saisu\AppData\Local\Temp\ipykernel_31092\869843029.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=qa.run(query)


The actors in *Inception* include Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, and Ken Watanabe.
